# The Guardian API

In the `05_web_scraping_beautiful_soup.ipynb` notebook, we saw examples on how BeautifulSoup can be used 
to parse messy HTML, to extract information, and to act as a rudimentary web crawler. 
We used The Guardian as an illustrative example about how this can be achieved. 
The reason for choosing The Guardian was because they provide a REST API to their servers. 
With the REST API it is possible to perform specific queries on their servers, and to receive 
current information from their servers according to their API guide (ie in JSON)

http://open-platform.theguardian.com/

In order to use their API, you will need to register for an API key. 
At the time of writing (Jan 28, 2020) this was an automated process that can be completed at 

https://bonobo.capi.gutools.co.uk/register/developer

On registration you will receive an API key which will look like: 303qwe2k-xxxx-xxxx-xxxx-eff86a248059

The API is documented here: 

http://open-platform.theguardian.com/documentation/

and Python bindings to their API are provided by The Guardian here

https://github.com/prabhath6/theguardian-api-python

and these can easily be integrated into a web-crawler based on API calls, rather than being based 
on HTML parsing, etc. 

We use four parameters in our queries here: 

1. `section`: the section of the newspaper that we are interested in querying. In this case we will look at 
the technology section 

2. `order-by`: We have specified that the newest items should be closer to the front of the query list 

3. `api-key`: In this notebook, the api-key is left as `test` (works here), but for *real* deployment of such a spider an API key obtained from Guardian should be specified. For the lab tasks, you should replace `test` API key with your personal API key. 

4. `page-size`: The number of results to return. 

In [251]:
from __future__ import print_function

import requests 
import json 
import re

# Inspect all sections and search for technology-based sections

In [252]:
url = 'https://content.guardianapis.com/search?api-key=aeff7dbf-1329-4a39-98e2-e42c822ab954'
req = requests.get(url)
src = req.text 

In [253]:
json.loads(src)['response']['status']

'ok'

In [254]:
sections = json.loads(src)['response']

print(sections.keys())

dict_keys(['status', 'userTier', 'total', 'startIndex', 'pageSize', 'currentPage', 'pages', 'orderBy', 'results'])


In [255]:
sections['results']

[{'id': 'us-news/live/2021/feb/05/joe-biden-donald-trump-impeachment-covid-coronavirus-marjorie-taylor-greene-live-updates',
  'type': 'liveblog',
  'sectionId': 'us-news',
  'sectionName': 'US news',
  'webPublicationDate': '2021-02-05T23:46:39Z',
  'webTitle': 'House passes budget resolution, paving way for Covid relief – US politics live',
  'webUrl': 'https://www.theguardian.com/us-news/live/2021/feb/05/joe-biden-donald-trump-impeachment-covid-coronavirus-marjorie-taylor-greene-live-updates',
  'apiUrl': 'https://content.guardianapis.com/us-news/live/2021/feb/05/joe-biden-donald-trump-impeachment-covid-coronavirus-marjorie-taylor-greene-live-updates',
  'isHosted': False,
  'pillarId': 'pillar/news',
  'pillarName': 'News'},
 {'id': 'world/2021/feb/05/all-uk-arrivals-to-face-double-testing-regime-government-to-declare',
  'type': 'article',
  'sectionId': 'world',
  'sectionName': 'World news',
  'webPublicationDate': '2021-02-05T23:37:13Z',
  'webTitle': 'All UK arrivals to face d

In [256]:
json.dumps(sections['results'][0], indent=2)

'{\n  "id": "us-news/live/2021/feb/05/joe-biden-donald-trump-impeachment-covid-coronavirus-marjorie-taylor-greene-live-updates",\n  "type": "liveblog",\n  "sectionId": "us-news",\n  "sectionName": "US news",\n  "webPublicationDate": "2021-02-05T23:46:39Z",\n  "webTitle": "House passes budget resolution, paving way for Covid relief \\u2013 US politics live",\n  "webUrl": "https://www.theguardian.com/us-news/live/2021/feb/05/joe-biden-donald-trump-impeachment-covid-coronavirus-marjorie-taylor-greene-live-updates",\n  "apiUrl": "https://content.guardianapis.com/us-news/live/2021/feb/05/joe-biden-donald-trump-impeachment-covid-coronavirus-marjorie-taylor-greene-live-updates",\n  "isHosted": false,\n  "pillarId": "pillar/news",\n  "pillarName": "News"\n}'

In [257]:
for result in sections['results']: 
    if 'tech' in result['id'].lower(): 
        print(result['webTitle'], result['apiUrl'])

# Manual query on whole API

In [258]:
# Specify the arguments
args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': 'aeff7dbf-1329-4a39-98e2-e42c822ab954', 
    'page-size': '100',
    'q' : 'privacy%20AND%20data'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [259]:
print('Number of byes received:', len(src))

Number of byes received: 60265


In [260]:
json.loads(src)

{'response': {'status': 'ok',
  'userTier': 'developer',
  'total': 3319,
  'startIndex': 1,
  'pageSize': 100,
  'currentPage': 1,
  'pages': 34,
  'orderBy': 'newest',
  'results': [{'id': 'technology/2021/feb/02/apple-iphone-update-solves-problem-of-unlocking-faceid-in-a-mask',
    'type': 'article',
    'sectionId': 'technology',
    'sectionName': 'Technology',
    'webPublicationDate': '2021-02-02T16:44:37Z',
    'webTitle': 'iPhone update lets Apple Watch users unlock Face ID in a mask',
    'webUrl': 'https://www.theguardian.com/technology/2021/feb/02/apple-iphone-update-solves-problem-of-unlocking-faceid-in-a-mask',
    'apiUrl': 'https://content.guardianapis.com/technology/2021/feb/02/apple-iphone-update-solves-problem-of-unlocking-faceid-in-a-mask',
    'isHosted': False,
    'pillarId': 'pillar/news',
    'pillarName': 'News'},
   {'id': 'world/2021/jan/28/apple-and-facebook-at-odds-over-privacy-move-that-will-hit-online-ads',
    'type': 'article',
    'sectionId': 'techno

The API returns JSON, so we parse this using the in-built JSON library. 
The API specifies that all data are returned within the `response` key, even under failure. 
Therefore, I have immediately descended to the response field 

In [261]:
response = json.loads(src)['response']
response.keys()

dict_keys(['status', 'userTier', 'total', 'startIndex', 'pageSize', 'currentPage', 'pages', 'orderBy', 'results'])

# task6

In [262]:
tech_section = response['results']

# status 
print("status:", response['status'])

# userTier 
print("userTier:", response['userTier'])

# total 
print("total:", response['total'])

# startIndex 
print("startIndex:", response['startIndex'])

# List the page size 
print("pagesize: ", response['pageSize'])

# currentPage
print("currentPage:", response['currentPage'])

# number of pages 
print("pages:", response['pages'])

# orderBy  
print("orderBy:", response['orderBy'])

# print(response['results'])

status: ok
userTier: developer
total: 3319
startIndex: 1
pagesize:  100
currentPage: 1
pages: 34
orderBy: newest


# task7

In [263]:
for result in response['results']: 
    if 'privacy' in result['id'].lower(): 
        print("PRIVACY")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
        
    if 'whatsapp' in result['id'].lower(): 
        print("WHATSAPP")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
        
    if 'signal' in result['id'].lower(): 
        print("SIGNAL")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
        
    if '-ai-' in result['id'].lower(): 
        print("AI")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
        
    if 'artificial intelligence' in result['id'].lower(): 
        print("Artificial intelligence")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
        

PRIVACY
id:  world/2021/jan/28/apple-and-facebook-at-odds-over-privacy-move-that-will-hit-online-ads
webTitle:  Apple and Facebook at odds over privacy move that will hit online ads
URL:  https://content.guardianapis.com/world/2021/jan/28/apple-and-facebook-at-odds-over-privacy-move-that-will-hit-online-ads

WHATSAPP
id:  technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing
webTitle:  UK regulator to write to WhatsApp over Facebook data sharing
URL:  https://content.guardianapis.com/technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing

PRIVACY
id:  technology/2021/jan/25/google-announces-plan-to-tackle-privacy-issues-in-online-advertising
webTitle:  Google announces plan to tackle privacy issues in online advertising
URL:  https://content.guardianapis.com/technology/2021/jan/25/google-announces-plan-to-tackle-privacy-issues-in-online-advertising

WHATSAPP
id:  technology/2021/jan/24/whatsapp-loses-millions-of-users-after-ter

# task7-a

In [264]:
# Specify the arguments
args = {
    'section': 'business', 
    'order-by': 'newest', 
    'api-key': 'aeff7dbf-1329-4a39-98e2-e42c822ab954', 
    'page-size': '100',
    'q' : 'privacy%20AND%20data'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [265]:
response = json.loads(src)['response']
response.keys()

dict_keys(['status', 'userTier', 'total', 'startIndex', 'pageSize', 'currentPage', 'pages', 'orderBy', 'results'])

In [266]:
for result in response['results']: 
    if 'stock' in result['id'].lower(): 
        print("STOCK")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
    
    if 'squeeze' in result['id'].lower(): 
        print("SQUEEZE")
        print("id: ", result['id'])
        print("webTitle: ", result['webTitle'])
        print("URL: ", result['apiUrl'])
        print()
        

STOCK
id:  business/live/2020/jun/24/asia-pacific-stock-markets-business-confidence-imf-growth-recession-business-live
webTitle:  European and US stock markets fall amid Covid-19 and trade fears - as it happened
URL:  https://content.guardianapis.com/business/live/2020/jun/24/asia-pacific-stock-markets-business-confidence-imf-growth-recession-business-live

STOCK
id:  business/2019/sep/14/technology-stock-market-ipos-2019-uber-lyft-slack-pinterest
webTitle:  Floating or falling? Tech companies that made stock market debuts in 2019
URL:  https://content.guardianapis.com/business/2019/sep/14/technology-stock-market-ipos-2019-uber-lyft-slack-pinterest

STOCK
id:  business/2019/jun/03/us-tech-stocks-alphabet-google-antitrust-investigation
webTitle:  US tech stocks slide as Google, Facebook and Apple fear antitrust investigations
URL:  https://content.guardianapis.com/business/2019/jun/03/us-tech-stocks-alphabet-google-antitrust-investigation



# task7-b

In [267]:
# 이거는 잘 모르겠다.

# task8 and task9

In [268]:
# Specify the arguments
args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': 'aeff7dbf-1329-4a39-98e2-e42c822ab954', 
    'page-size': '100',
    'q' : 'privacy%20AND%20data'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [269]:
response = json.loads(src)['response']
response.keys()

dict_keys(['status', 'userTier', 'total', 'startIndex', 'pageSize', 'currentPage', 'pages', 'orderBy', 'results'])

In [270]:
def word_counter(text):
    return len(text.split())

In [271]:
def unique_word_counter(text):
    return len(set(text.split()))

In [272]:
def occurrence_checker(text):
    # Create an empty dictionary 
    d = dict() 
  
    # Split the line into words 
    words = text.split(" ") 

    # Iterate over each word in line 
    for word in words: 
        # Check if the word is already in dictionary 
        if word in d: 
            # Increment count of word by 1 
            d[word] = d[word] + 1
        else: 
            # Add the word to dictionary with count 1 
            d[word] = 1
        
    return d.keys(), d.values()

In [273]:
# cleans tags as well as redundant ' " and etc 
def tag_cleaner(body_text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    text  = re.sub(clean, '', body_text)
    
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    answer = ''.join(filter(whitelist.__contains__, text))
    
    return answer

In [274]:
def headline_and_text_finder(api_id):
    base_url = "https://content.guardianapis.com/search?"
    search_string = "ids=%s&api-key=%s&show-fields=headline,body" %(api_id, 'aeff7dbf-1329-4a39-98e2-e42c822ab954')
    url = base_url + search_string
    
    req = requests.get(url) 
    src = req.text
    response = json.loads(src)['response']
    
    return response['results'][0]['fields']['headline'], response['results'][0]['fields']['body']

In [275]:
def info_printer(interested_word, result, api_id, words):
    print(f"\nInterested word: {interested_word}")
    print("\nId: ", result['id'])
    print("\nWebTitle: ", result['webTitle'])
    print("\nURL: ", result['apiUrl'])
    headline, body = headline_and_text_finder(api_id)
    
    # clean tags in the body text
    body = tag_cleaner(body)
    body = body.lower()
    
    flag = 0
    for word in words: 
        if word in body.lower():
            print(f"Not printing because it contains: {word} !")
            flag = 1
            break
    
    if flag == 0:
        print("\n<Headline>\n", headline)
        print("\n<Body>\n", body)
    else: # flag == 1
        pass
    
    print("\nNumber of words in this article: \n", \
          word_counter(body))
    
    keys, values = occurrence_checker(body)
    print("\nNumber of unique words in this article: \n", \
          len(keys))
    print("===========================================")
    
    return api_id, keys, values

In [276]:
apiid_list = list()
ks_list = list()
vs_list = list()

for result in response['results']: 
    
    if 'privacy' in result['id'].lower(): 
        apiid, ks, vs = info_printer('privacy', result, result['id'], ['AI', 'artificial intellience'])
        apiid_list.append(apiid)
        ks_list.append(ks)
        vs_list.append(vs)
        
    if 'whatsapp' in result['id'].lower(): 
        apiid, ks, vs = info_printer('whatsapp', result, result['id'], ['AI', 'artificial intellience'])
        apiid_list.append(apiid)
        ks_list.append(ks)
        vs_list.append(vs)
        
    if 'signal' in result['id'].lower(): 
        apiid, ks, vs = info_printer('signal', result, result['id'], ['AI', 'artificial intellience'])
        apiid_list.append(apiid)
        ks_list.append(ks)
        vs_list.append(vs)


Interested word: privacy

Id:  world/2021/jan/28/apple-and-facebook-at-odds-over-privacy-move-that-will-hit-online-ads

WebTitle:  Apple and Facebook at odds over privacy move that will hit online ads

URL:  https://content.guardianapis.com/world/2021/jan/28/apple-and-facebook-at-odds-over-privacy-move-that-will-hit-online-ads

<Headline>
 Apple and Facebook at odds over privacy move that will hit online ads

<Body>
 apples chief executive tim cook has launched his strongest attack on facebook yet as the two companies face off over apples plans for new privacy features that would severely limit online advertising speaking to the computers privacy and data protection conference on data privacy day cook defended apples decision to introduce the features called app tracking transparency att that setting coming to iphones in early spring will for the first time require apps to ask for users permission in order to track them around the web apples att plans have brought stark criticism from


<Headline>
 WhatsApp loses millions of users after terms update

<Body>
 a poorly explained update to its terms of service has pushed whatsapp users to adopt alternative services such as signal and telegram in their millions the exodus was so large that whatsapp has been forced to delay the implementation of the new terms which had been slated for  february and run a damage limitation campaign to explain to users the changes they were making over the first three weeks of january signal has gained  million users globally according to figures shared by the uk parliaments home affairs committee and telegram has gained  million in both cases the increase appears to have come at whatsapps expense data tracked by the analytics firm app annie shows whatsapp falling from the eighth most downloaded app in the uk at the beginning of the month to the rd by  january by contrast signal wasnt even in the top  apps in the uk on  january yet by  january it was the most downloaded app in the country n


<Headline>
 Is it time to leave WhatsApp – and is Signal the answer?

<Body>
 earlier this month whatsapp issued a new privacy policy along with an ultimatum accept these new terms or delete whatsapp from your smartphone but the new privacy policy wasnt particularly clear and it was widely misinterpreted to mean whatsapp would be sharing more sensitive personal data with its parent company facebook unsurprisingly it prompted a fierce backlash with many users threatening to stop using the service whatsapp soon issued a clarification explaining that the new policy only affects the way users accounts interact with businesses ie not with their friends and does not mandate any new data collection the messaging app also delayed the introduction of the policy by three months crucially whatsapp said the new policy doesnt affect the content of your chats which remain protected by endtoend encryption  the gold standard of security that means no one can view the content of messages even whatsapp


<Headline>
 Should you keep using WhatsApp? Plus five tips to start the year with your digital privacy intact

<Body>
 if you use the popular messaging service whatsapp you may have noticed a popup message in recent days asking you to accept the services new terms and conditions by  february in order to continue using it the update has prompted calls for users to leave the popular messaging service in favour of alternatives such as signal and telegram and on friday a legal challenge on privacy grounds was filed against whatsapp in india the services biggest market telegram ceo pavel durov has reported an influx of  million global users to the rival service since the announcement was made    use signalmdash elon musk elonmusk january      but what do the new terms and conditions mean for you we asked former mostwanted hacker turned security consultant kevin mitnick which messaging app he prefers  and to share his tips to set yourself up for a cybersafe   i prefer signal because i know 


<Headline>
 Apple accuses Facebook of 'disregard for user privacy'

<Body>
 apple has criticised facebook for trying to collect as much data as possible from users saying it will push ahead with its planned launch of a new privacy feature despite objections from the advertising industry the companys director of global privacy jane horvath made the criticism in a letter to a coalition of privacy groups reassuring them that the feature which will require users to actively allow developers to track how they use other apps would still be launched we developed app tracking transparency for a single reason because we share your concerns about users being tracked without their consent and the bundling and reselling of data by advertising networks and data brokers horvath wrote she defended apples approach to targeted adverts which she said was based on demographic details rather than user tracking facebook and others have a very different approach to targeting horvath wrote not only do they 


<Headline>
 'I don't care': young TikTokers unfazed by US furor over data collection

<Body>
 mauren sparrow downloaded tiktok in march to pass the time during lockdown since then shes posted tutorials on crafting and videos of her two cats calcifer and jiji some of which have accrued millions of views and likes but with the chineseowned app now under fire over data privacy concerns sparrow  and other young users have reacted with a resounding shrug    im so used to all social networks having my data that i feel its just the price i have to pay to connect with others  mauren sparrow tiktok user     i dont really care that these corporations have my data as long as i know they have it sparrow says at this point im so used to all social networks having all of my data that i feel its just the price i have to pay to connect with others tiktoks future has been in flux for weeks after the us secretary of state mike pompeo hinted at a potential ban in early july most recently donald trump ap


<Headline>
 US judge: WhatsApp lawsuit against Israeli spyware firm NSO can proceed

<Body>
 an israeli company whose spyware has been used to target journalists in india politicians in spain and human rights activists in morocco may soon be forced to divulge information about its government clients and practices after a judge in california ruled that a lawsuit against the company could proceed nso group was sued by whatsapp which is owned by facebook last year after the popular messaging app accused the company of sending malware to  of its users over a twoweek period and targeting their mobile phones in a ruling that the case could proceed in a us district court in california judge phyllis hamilton said she had not been entirely persuaded by nso groups argument that it had no role in the targeting of whatsapps users instead judge hamilton said it appeared that nso group retained some role in the targeting of individuals even if it was at the direction of their customers the case wil


<Headline>
 What's wrong with WhatsApp

<Body>
 in the spring as the virus swept across the world and billions of people were compelled to stay at home the popularity of one social media app rose more sharply than any other by late march usage of whatsapp around the world had grown by  in spain where the lockdown was particularly strict it rose by  in those early months whatsapp  which hovers neatly between the space of email facebook and sms allowing text messages links and photos to be shared between groups  was a prime conduit through which waves of news memes and mass anxiety travelled   related whats wrong with whatsapp  podcast    at first many of the new uses were heartening mutual aid groups sprung up to help the vulnerable families and friends used the app to stay close sharing their fears and concerns in real time yet by midapril the role that whatsapp was playing in the pandemic looked somewhat darker a conspiracy theory about the rollout of g which originated long before c


<Headline>
 UK’s facial recognition technology ‘breaches privacy rights’

<Body>
 automated facial recognition technology that searches for people in public places breaches privacy rights and will radically alter the way britain is policed the court of appeal has been told at the opening of a legal challenge against the use by south wales police of the mass surveillance system lawyers for the civil rights organisation liberty argued that it is also racially discriminatory and contrary to data protection laws in written submissions to the court dan squires qc who is acting for liberty and ed bridges a cardiff resident said that the south wales force had already captured the biometrics of  faces the overwhelming majority of whom are not suspected of any wrongdoing bridges  whose face was scanned while he was christmas shopping in cardiff in  and at a peaceful antiarms protest outside the citys motorpoint arena in  says the use of automatic facial recognition afr by south wales police ca

In [277]:
import pandas as pd

In [278]:
print(f"<Api Id>: {apiid_list[0]}")
list_of_tuples = list(zip(ks_list[0], vs_list[0]))  
    
df = pd.DataFrame(list_of_tuples, columns = ['words', 'occurence'])
df

<Api Id>: world/2021/jan/28/apple-and-facebook-at-odds-over-privacy-move-that-will-hit-online-ads


,words,occurence
0,apples,5
1,chief,2
2,executive,2
3,tim,1
4,cook,3
...,...,...
307,position,1
308,interfere,1
309,work,1
310,regularly,1


# Parsing the JSON

In [279]:
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']


# Verifying the status code

It is important to verify that the status message is `ok` before continuing - if it is not `ok` no 'real' data 
will have been received. 

In [280]:
assert response['status'] == 'ok'

# Listing the results 

The API standard states that the results will be found in the `results` field under the `response` field. 
Furthermore, the URLs will be found in the `webUrl` field, and the title will be found in the `webTitle` 
field. 

First let's look to see what a single result looks like in full, and then I will print a restricted 
set of parameters on the full set of results .

In [281]:
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/technology/2021/feb/02/apple-iphone-update-solves-problem-of-unlocking-faceid-in-a-mask",
  "id": "technology/2021/feb/02/apple-iphone-update-solves-problem-of-unlocking-faceid-in-a-mask",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2021-02-02T16:44:37Z",
  "webTitle": "iPhone update lets Apple Watch users unlock Face ID in a mask",
  "webUrl": "https://www.theguardian.com/technology/2021/feb/02/apple-iphone-update-solves-problem-of-unlocking-faceid-in-a-mask"
}


In [282]:
for result in response['results']: 
    print(result['webUrl'][:70], result['webTitle'][:20])

https://www.theguardian.com/technology/2021/feb/02/apple-iphone-update iPhone update lets A
https://www.theguardian.com/world/2021/jan/28/apple-and-facebook-at-od Apple and Facebook a
https://www.theguardian.com/technology/2021/jan/27/facebook-earnings-s Facebook CEO Mark Zu
https://www.theguardian.com/technology/2021/jan/26/uk-regulator-to-wri UK regulator to writ
https://www.theguardian.com/technology/2021/jan/26/grindr-fined-norway Grindr fined £8.6m i
https://www.theguardian.com/technology/2021/jan/25/google-announces-pl Google announces pla
https://www.theguardian.com/technology/2021/jan/24/whatsapp-loses-mill WhatsApp loses milli
https://www.theguardian.com/technology/2021/jan/24/is-it-time-to-leave Is it time to leave 
https://www.theguardian.com/technology/2021/jan/21/samsung-galaxy-s21- Samsung Galaxy S21 U
https://www.theguardian.com/technology/2021/jan/21/facebook-admits-enc Facebook admits encr
https://www.theguardian.com/technology/2021/jan/20/facebook-under-pres Facebook 